# Tarea 2 – ISIS 4221, Notebook I

## Paso 0: Importación de Librerías Necesarias

<span style="color: gray;">Añadí import statements para `nltk`.</span>

<span style="color: gray;">Añadí código para crear los directorios necesarios, si no existen.</span>

In [ ]:
import os
import json
from glob import glob
from importlib import import_module
from itertools import chain
from re import compile, DOTALL, findall, split
from collections import Counter, defaultdict

import nltk   # Para tokenizar mejor
from nltk.tokenize import sent_tokenize   # Para tokenizar mejor
import numpy as np
from pandas import DataFrame, Series
from tqdm import tqdm

nltk.download('punkt')   # Para usar con nltk

In [ ]:
# Define los directorios que se deben crear
ngram_models_dir = './data/ngram_models/'
train_test_dir = './data/train_test/'

# Crea los directorios si no existen
os.makedirs(ngram_models_dir, exist_ok=True)
os.makedirs(train_test_dir, exist_ok=True)

## Paso 1: Definición de las Funciones Necesarias

<span style="color: gray;">Sugerí crear `sentences` utilizando el tokenizador de oraciones de `nltk` para una mejor precisión (por ejemplo, para capturar signos de puntuación como "!" o "?" y no separar erróneamente oraciones como "e.g.".</span>

<span style="color: gray;">Sugerí usar una expresión regular diferentes con `numbers_re` para manejar números con comas (por ejemplo, 1,000) y decimales (por ejemplo, 1.50).</span>

<span style="color: gray;">Cambié el signo `==` por el signo `>=` en el generador que devuelve la función `create_ngrams` para garantizar que las oraciones tengan suficientes palabras (i.e., si la longitud de la oración es menor a `n`, no se generarán n-gramas).</span>

<span style="color: gray;">Añadí la función `save_ngram_model`.</span>

In [ ]:
def read_file(f: str) -> str:
    """
    Lee el contenido de un archivo y maneja posibles errores de codificación.

    Args:
        f (str): La ruta al archivo que se desea leer.

    Returns:
        str: El contenido del archivo como una cadena de texto.

    Notas:
        - La función intenta leer el archivo con codificación 'utf-8' por defecto.
        - Si ocurre un error de decodificación (UnicodeDecodeError), se reintenta con la codificación 'latin1'.
        - Este enfoque es útil cuando se manejan archivos de texto con codificaciones mixtas o desconocidas.
    """
    try:
        with open(f, 'r', encoding='utf-8') as file:
            txt = file.read()
    except UnicodeDecodeError:
        with open(f, 'r', encoding='latin1') as file:
            txt = file.read()
    
    return txt

def clean_text(txt: str) -> str:
    """
    Limpia el texto eliminando correos electrónicos, etiquetas HTML, caracteres no alfanuméricos,
    y otros elementos no deseados.

    Args:
        txt (str): El texto que se desea limpiar.

    Returns:
        str: El texto limpio y normalizado.

    Pasos:
        - Convertir el texto a minúsculas para normalizarlo.
        - Eliminar correos electrónicos, etiquetas HTML, y ciertas palabras clave usando expresiones regulares.
        - Reemplazar secuencias de puntos múltiples con un solo espacio.
        - Eliminar repeticiones de caracteres consecutivos (más de dos veces) que no sean necesarias.
        - Reemplazar guiones por espacios para evitar palabras concatenadas por guiones.
        - Sustituir signos de puntuación específicos (?, !, :) por un punto.
        - Eliminar caracteres no alfanuméricos, excepto puntos, espacios y saltos de línea.
        - Sustituir números por la palabra 'NUM' para normalizar las secuencias numéricas.
        - Reemplazar múltiples saltos de línea consecutivos con un solo punto.
        - Convertir saltos de línea individuales en espacios.
        - Reemplazar múltiples espacios consecutivos por un solo espacio.

    Notas:
        - El uso de expresiones regulares precompiladas mejora la eficiencia al aplicar la limpieza a múltiples textos.
    """

    # Compila las expresiones regulares fuera de la función para mejorar la eficiencia
    email_re = compile(r'\S*@\S*\s?|from: |re: |subject: |urllink|maxaxaxaxaxaxaxaxaxaxaxaxaxaxax')
    punctuation_re = compile(r'[?!:]')
    non_alphanumeric_re = compile(r'[^A-Za-z0-9. \n]')
    # Línea original de Raúl: numbers_re = compile(r'\b\d+\b')
    # Mi sugerencia:
    numbers_re = compile(r'\b\d{1,3}(,\d{3})*(\.\d+)?\b')
    multiple_newlines_re = compile(r'\n{2,}')
    single_newline_re = compile(r'\n')
    multiple_dots_re = compile(r'\.\.+')
    multiple_spaces_re = compile(r'\s+')
    multiple_char_repetition_re = compile(r'(.)\1{2,}')
    
    # Convierte el texto a minúsculas
    txt = txt.lower()

    # Elimina correos electrónicos, etiquetas dentro de <>, y ciertas palabras clave
    txt = email_re.sub('', txt)

    # Reemplaza secuencias de puntos múltiples con un solo espacio
    txt = multiple_dots_re.sub(' ', txt)

    # Elimina repeticiones de caracteres consecutivos (más de dos veces)
    txt = multiple_char_repetition_re.sub(r'\1', txt)

    # Reemplaza guiones por espacios
    txt = txt.replace('-', ' ')

    # Sustituye signos de puntuación específicos por un punto
    txt = punctuation_re.sub('.', txt)

    # Elimina caracteres no alfanuméricos excepto puntos, espacios y saltos de línea
    txt = non_alphanumeric_re.sub('', txt)

    # Sustituye números por 'NUM'
    txt = numbers_re.sub('NUM', txt)

    # Reemplaza múltiples saltos de línea consecutivos por un solo punto
    txt = multiple_newlines_re.sub('.', txt)

    # Convierte saltos de línea individuales en espacios
    txt = single_newline_re.sub(' ', txt)

    # Reemplaza múltiples espacios consecutivos por un solo espacio
    txt = multiple_spaces_re.sub(' ', txt)

    # Elimina espacios en blanco al inicio y final del texto
    return txt.strip()

def extract_and_process_text_from_xml(file_path: str) -> list:
    """
    Extrae y procesa el texto contenido entre etiquetas <post> en un archivo XML.

    Args:
        file_path (str): La ruta al archivo XML del cual se extraerá el texto.

    Returns:
        list: Una lista de diccionarios, donde cada diccionario representa una oración extraída y procesada
              con las claves:
              - 'text': La oración procesada.
              - 'length': La longitud de la oración en número de palabras.

    Notas:
        - La función primero lee el archivo XML utilizando `read_file`.
        - Luego extrae el contenido de todas las etiquetas <post> usando expresiones regulares.
        - Cada fragmento de texto extraído se limpia utilizando la función `clean_text`.
        - El texto limpio se divide en oraciones, que se formatean y almacenan en una lista de diccionarios.
        - Cada oración es rodeada por etiquetas <s> y </s> para indicar su inicio y fin.
        - Las oraciones que contienen una sola palabra o están vacías se descartan.
    """
    # Lee el contenido del archivo XML
    xml_content = read_file(file_path)
    
    # Extrae el contenido entre las etiquetas <post> y </post> utilizando expresiones regulares
    post_matches = findall(r'<post>(.*?)</post>', xml_content, DOTALL)
    
    df_rows = []
    
    for post in post_matches:
        # Limpia el texto extraído utilizando la función clean_text
        cleaned_post = clean_text(post.strip())
        
        # Divide el texto limpio en oraciones
        # Línea original de Raúl: sentences = [f'<s> {s.strip()} </s>' for s in split(r'\.\s*', cleaned_post) if len(s.strip().split()) > 1]
        # Mi sugerencia:
        sentences = [f'<s> {s.strip()} </s>' for s in sent_tokenize(cleaned_post) if len(s.strip().split()) > 1]
        
        # Crea un diccionario para cada oración con su texto y longitud
        df_rows.extend([{
            'text': s,
            'source': file_path,
            'length': len(s.split())
        } for s in sentences])

    return df_rows

def create_ngrams(sentence: str, n: int, unique_tokens: dict = None) -> list:
    """
    Genera n-gramas a partir de una oración dada.

    Args:
        sentence (str): La oración de entrada de la cual se generarán los n-gramas.
        n (int): La longitud de los n-gramas a generar.
        unique_tokens (dict, opcional): Un diccionario que mapea ciertos tokens a un valor único,
            típicamente utilizado para reemplazar tokens poco frecuentes por un marcador como '<UNK>'.
            Por defecto es None.

    Returns:
        list: Una lista de n-gramas, donde cada n-grama se representa como una tupla de palabras.

    Notas:
        - Si se proporciona `unique_tokens`, cualquier palabra en la oración que aparezca en 
          `unique_tokens` será reemplazada según el mapeo en `unique_tokens`.
        - La función devuelve n-gramas en forma de un generador para ahorrar memoria, 
          especialmente útil al procesar grandes corpus.
    """
    words = sentence.split()  # Divide la oración en palabras
    
    if unique_tokens:
        # Genera n-gramas usando un generador para reemplazar las palabras según el diccionario unique_tokens
        return (tuple([unique_tokens.get((w,), w) for w in words[i:i+n]]) 
                for i in range(len(words) - n + 1))
    
    # Genera los n-gramas usando un generador sin realizar reemplazos
    return (tuple(words[i:i+n]) for i in range(len(words) - n + 1) if len(words[i:i+n]) == n)

def create_ngram_model(n_gram: int, text_corpus: Series)-> tuple[Counter[int], defaultdict[int]]:
    """
    Crea un modelo de n-gramas a partir de un corpus de texto.

    Args:
        n_gram (int): El tamaño de los n-gramas a generar.
        text_corpus (pd.Series): Una serie de Pandas que contiene el corpus de texto, 
                                 donde cada entrada es una oración o texto.

    Returns:
        tuple: Dos elementos:
            - ngram_counts (Counter): Un contador que almacena las frecuencias de los n-gramas en el corpus.
            - final_unigram (defaultdict): Un diccionario con los (n-1)-gramas más frecuentes y su conteo, 
                                           con un marcador especial `<UNK>` para los menos frecuentes.

    Notas:
        - La función primero cuenta los (n-1)-gramas para identificar los tokens únicos que serán 
          reemplazados por `<UNK>`.
        - Luego, se cuentan los n-gramas completos utilizando los reemplazos identificados.
    """    

    # Cuenta las frecuencias de (n-1)-gramas en el corpus de texto
    unigram_counts = Counter(chain.from_iterable(
        text_corpus.apply(lambda x: create_ngrams(x, n=n_gram-1, unique_tokens=None))
    ))

    final_unigram = defaultdict(int)
    unique_tokens = defaultdict(int)

    # Identifica tokens únicos y construir el diccionario final de 'unigrama'
    for ngram, count in unigram_counts.items():
        if '' in ngram:
            continue
        if count < 2:
            unique_tokens[ngram] = ('<UNK>',) * (n_gram-1)
        else:
            final_unigram[ngram] = count

    final_unigram[('<UNK>',)*(n_gram-1)] = len(unique_tokens)

    # Cuenta las frecuencias de los n-gramas completos usando los reemplazos identificados
    ngram_counts = Counter(chain.from_iterable(
        text_corpus.apply(lambda x: create_ngrams(x, n=n_gram, unique_tokens=unique_tokens))
    ))

    return ngram_counts, final_unigram

def save_ngram_model(ngram_counts, model_name):
    """
    Guarda el modelo de n-gramas en un archivo JSON.

    Args:
        ngram_counts (Counter): Los n-gramas y sus conteos.
        model_name (str): Nombre del archivo donde se guardará el modelo.
    """
    with open(f'./data/ngram_models/{model_name}.json', 'w') as f:
        json.dump(ngram_counts, f)
    print(f"{model_name} saved.")

<span style="color: red;">No entiendo para qué funciona o cómo debo usar la siguiente función. En mi extensión del código de Raúl no la uso.</span>

In [ ]:
def estimate_probability(token_text: list[str], n_gram: int, 
                         final_unigram: dict, ngram_counts: dict) -> float:
    """
    Estima la probabilidad de un n-grama dado usando un modelo de n-gramas.

    Args:
        token_text (list[str]): La secuencia de tokens para la cual se desea estimar la probabilidad.
        n_gram (int): El tamaño del n-grama.
        final_unigram (dict): El diccionario que contiene las frecuencias de los (n-1)-gramas.
        ngram_counts (dict): El diccionario que contiene las frecuencias de los n-gramas.

    Raises:
        ValueError: Si el número de tokens en `token_text` no coincide con `n_gram`.

    Returns:
        float: La probabilidad estimada del n-grama dado.

    Notas:
        - La función utiliza suavizado de Laplace para evitar probabilidades de cero.
        - Si el prefijo (n-1)-grama no se encuentra en `final_unigram`, se reemplaza con (`<UNK>`,)* (n_gram-1).
    """    
    
    if len(token_text) != n_gram:
        raise ValueError(f'El texto de entrada debe tener {n_gram} tokens')
    
    # Reemplaza el prefijo con `<UNK>` si no se encuentra en final_unigram
    if not final_unigram.get(tuple(token_text[:n_gram-1])):
        token_text = list(token_text)
        token_text[:n_gram] = ('<UNK>',) * (n_gram-1)
        token_text = tuple(token_text)

    # Suavizado de Laplace
    # Probabilidad del (n-1)-grama más tamaño del vocabulario
    p_wi = final_unigram.get(tuple(token_text[:n_gram-1]), 0) + len(final_unigram)
    
    # Probabilidad del n-grama completo dado su (n-1)-grama
    p_w_wi = (ngram_counts.get(tuple(token_text), 1)) / p_wi
        
    return p_w_wi

<span style="color: red;">No entiendo para qué funciona o cómo debo la función de arriba. En mi extensión del código de Raúl no la uso.</span>

## Paso 2: Procesamiento y Almacenamiento de 20 Newsgroups (20N)

<span style="color: gray;">Añadí (i) una columna `category` que extrae el nombre del subdirectorio para que cada documento quede categorizado de acuerdo con el *newsgroup* del que proviene, y (ii) una descripción en el comentario inicial sobre la organización de los archivos dentro de subdirectorios que representan las diferentes categorías (*newsgroups*).</span>

<span style="color: gray;">Sugerí crear `sentences` utilizando el tokenizador de oraciones de `nltk` para una mejor precisión (por ejemplo, para capturar signos de puntuación como "!" o "?" y no separar erróneamente tokens como "e.g.".</span>

<span style="color: gray;">Añadí código para guardar con los nombres correctos los conjuntos de entrenamiento y prueba.</span>

<span style="color: gray;">Añadí código para generar y guardar con los nombres correctos los modelos de N-gramas.</span>

In [ ]:
# Obtiene una lista de archivos en el directorio './raw_data/20news-18828/' que contiene subdirectorios para cada newsgroup
files_20n = glob('./raw_data/20news-18828/*/*')

# Cuenta el número total de archivos encontrados
print(f"Número total de archivos encontrados: {len(files_20n)}")

In [ ]:
# Inicializa una lista para almacenar las filas procesadas
df_news_rows = []

# Procesa cada archivo encontrado
for f in tqdm(files_20n):
    # Extrae la categoría (nombre del subdirectorio) a partir de la ruta del archivo
    category = os.path.basename(os.path.dirname(f))
    
    # Lee el contenido del archivo
    txt = read_file(f)
    
    # Limpia el texto leído
    txt_cln = clean_text(txt)
    
    # Divide el texto limpio en oraciones, añadiendo etiquetas de inicio y fin de oración
    # Línea original de Raúl: sentences = [f'<s> {s.strip()} </s>' for s in re.split(r'\.\s*', txt_cln) if len(s.strip().split()) > 1]
    # Mi sugerencia:
    sentences = [f'<s> {s.strip()} </s>' for s in sent_tokenize(txt_cln) if len(s.strip().split()) > 1]
    
    # Crea un diccionario para cada oración con el texto, la fuente del archivo, la categoría y la longitud de la oración
    df_news_rows.extend([{
        'text': s,
        'source': f,
        'category': category,  # Añade la categoría como una columna adicional
        'length': len(s.split())
    } for s in sentences])

# Crea un DataFrame a partir de las filas procesadas
df_news = pd.DataFrame(df_news_rows)

# Muestra el DataFrame resultante
print(df_news.head())

# Guarda el DataFrame en un archivo Parquet para su almacenamiento eficiente
df_news.to_parquet('./data/20news.parquet', index=False)

In [ ]:
# Divide aleatoriamente el DataFrame en conjuntos de entrenamiento y prueba (80% entrenamiento, 20% prueba)
df_news_train = df_news.sample(frac=0.8, random_state=42)
df_news_test = df_news.drop(df_news_train.index)

# Guarda los conjuntos de entrenamiento y prueba en archivos Parquet separados
df_news_train.to_parquet('./data/train_test/20N_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba_training.parquet', index=False)
df_news_test.to_parquet('./data/train_test/20N_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba_testing.parquet', index=False)

In [ ]:
# Genera los modelos de N-gramas
unigram_counts_20n, unigram_20n = create_ngram_model(1, df_news['text'])
bigram_counts_20n, bigram_20n = create_ngram_model(2, df_news['text'])
trigram_counts_20n, trigram_20n = create_ngram_model(3, df_news['text'])

# Guarda los modelos de N-gramas
save_ngram_model(unigram_counts_20n, './data/ngram_models/20N_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba_unigrams')
save_ngram_model(bigram_counts_20n, './data/ngram_models/20N_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba_bigrams')
save_ngram_model(trigram_counts_20n, './data/ngram_models/20N_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba_trigrams')

## Paso 3: Procesamiento y Almacenamiento de Blog Authorship Corpus (BAC)

<span style="color: gray;">Añadí código para guardar con los nombres correctos los conjuntos de entrenamiento y prueba.</span>

<span style="color: gray;">Añadí código para generar y guardar con los nombres correctos los modelos de N-gramas.</span>

In [ ]:
# Obtiene una lista de archivos XML en el directorio './raw_data/blogs/'
files_bac = glob('./raw_data/blogs/*')

# Cuenta el número total de archivos encontrados
len(files_bac)

In [ ]:
# Inicializa una lista para almacenar las filas procesadas
df_bac_rows = []

# Procesa cada archivo XML encontrado
for f in tqdm(files_bac):
    # Extraer y procesar el texto del archivo XML, y agregar las filas al DataFrame
    df_bac_rows.extend(extract_and_process_text_from_xml(f))

# Crea un DataFrame a partir de las filas procesadas
df_bac = DataFrame(df_bac_rows)

# Muestra el DataFrame resultante
print(df_bac.head())

# Guarda el DataFrame en un archivo Parquet para su almacenamiento eficiente
df_bac.to_parquet('./data/bac.parquet', index=False)

In [ ]:
# Divide el DataFrame en conjuntos de entrenamiento (80%) y prueba (20%) de manera aleatoria
df_bac_train = df_bac.sample(frac=0.8, random_state=42)
df_bac_test = df_bac.drop(df_bac_train.index)

# Guarda los conjuntos de entrenamiento y prueba en archivos Parquet separados
df_bac_train.to_parquet('./data/train_test/BAC_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba_training.parquet', index=False)
df_bac_test.to_parquet('./data/train_test/BAC_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba_testing.parquet', index=False)

In [ ]:
# Genera los modelos de N-gramas
unigram_counts_bac, unigram_bac = create_ngram_model(1, df_bac['text'])
bigram_counts_bac, bigram_bac = create_ngram_model(2, df_bac['text'])
trigram_counts_bac, trigram_bac = create_ngram_model(3, df_bac['text'])

# Guarda los modelos de N-gramas
save_ngram_model(unigram_counts_bac, './data/ngram_models/BAC_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba_unigrams')
save_ngram_model(bigram_counts_bac, './data/ngram_models/BAC_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba_bigrams')
save_ngram_model(trigram_counts_bac, './data/ngram_models/BAC_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba_trigrams')